<h1> Respostas para Dados - Q2 - Notebook </h1>

Olá leitor, este notebook tem como objetivo apresentar a solução da Q1 da cadeira de dados.

Então vamos lá!

<h2> Tabela guida <h2>

1. [ Parte 1 - Importanto bibliotecas. ](#1)
    
2. [ Parte 2 - Tratando o arquivo ](#2)
    
3. [ Parte 3 - Transformando txt em um df 2 ](#3)
    
4. [ Parte 4 - Tratando o df](#4)
    
5. [ Parte 5 - Tranformando df para o o formato json pedido ](#5)

<a name="1"></a>
<h2> Parte 1 - Importando bibliotecas </h2>

Primeiro vamos importar as bibliotecas utilizadas neste projeto:

In [159]:
import pandas as pd
import re 
from datetime import datetime, timedelta
import json

In [160]:
teste = open('DP.txt').read()

<a name="2"></a>
<h2> Parte 2 - Tratando o arquivo </h2>

Vamos tratar o arquivo passado e depois gravar no diretório com o nome `DP_TRATADO.txt`

In [161]:
tratamento1 = re.sub('&', '',teste)
tratamento2 = re.sub('X', '',tratamento1)
tratamento3 = re.sub('CARGA', '', tratamento2)
tratamento4 = re.sub('DP', '', tratamento3)
tratamento5 = re.sub('hf', '', tratamento4)
tratamento6 = re.sub('\n\n', '', tratamento5)
open('DP_Tratado.txt','w').write(tratamento6)

7336

<a name="3"></a>
<h2> Parte 3 - Transformando txt em um df </h2>

Vamos abrir o arquivo como um df para facilitar o trabalho.

In [162]:
df = pd.read_csv('DP_Tratado.txt',delimiter="\s+", index_col=False)

In [163]:
df

,ss,di,hi,m,df,m.1,Demanda
0,1,2,0,0,F,36703.0,NaN
1,1,2,0,1,F,35673.0,NaN
2,1,2,1,0,F,34527.0,NaN
3,1,2,1,1,F,33996.0,NaN
4,1,2,2,0,F,33526.0,NaN
...,...,...,...,...,...,...,...
216,4,4,0,0,F,5944.0,NaN
217,4,4,7,0,F,5618.0,NaN
218,4,4,10,0,F,6125.0,NaN
219,4,4,22,0,F,6504.0,NaN


<a name="4"></a>
<h2> Parte 4 - Tratando o df </h2>

Vamos tratar o df para depois fabricar o arquivo final

In [164]:
#ACERTANDO COLUNAS

df.drop('Demanda', axis=1, inplace=True)
df.rename(columns={'m.1':'Demanda'}, inplace=True)

In [165]:
#Mapeando portas segundo legenda

ports = {1: 'seco', 2: "s", 3: "ne", 4:"n", 11:"imperatriz"}
df['ss'] = df['ss'].map(ports)

ports = {0:0, 1: 30}
df['m'] = df['m'].map(ports)

In [166]:
# Mudando os tipos da coluna para string para realizar a conversao da hora

df['hi'] = df['hi'].apply(str)
df['m'] = df['m'].apply(str)
df['hora'] = df['hi'] + ':' + df['m']

In [167]:
# Excluindo coluna que não seram mais usadas

df.drop('df', axis=1, inplace=True)
df.drop(['hi','m'], axis=1, inplace=True)

In [168]:
# Prepara e acrescenta uma coluna datas em df

data_partida = datetime.fromisoformat("2021-09-25")
datas = [(data_partida + timedelta(days = dias-1)) for dias in df['di']]
df['datas'] = datas

In [169]:
# Prepara e acrescenta uma coluna horas em df e exclui coluna hora

horas = [(datetime.strptime(hr,"%H:%M").time()) for hr in df['hora']]
df['horas'] = horas
df.drop('hora', axis=1, inplace=True)
df

,ss,di,Demanda,datas,horas
0,seco,2,36703.0,2021-09-26,00:00:00
1,seco,2,35673.0,2021-09-26,00:30:00
2,seco,2,34527.0,2021-09-26,01:00:00
3,seco,2,33996.0,2021-09-26,01:30:00
4,seco,2,33526.0,2021-09-26,02:00:00
...,...,...,...,...,...
216,n,4,5944.0,2021-09-28,00:00:00
217,n,4,5618.0,2021-09-28,07:00:00
218,n,4,6125.0,2021-09-28,10:00:00
219,n,4,6504.0,2021-09-28,22:00:00


In [170]:
df['datas'] = df['datas'].astype(str)
df['horas'] = df['horas'].astype(str)
df['datas/horas'] = df['datas'] +" "+ df['horas']
df.drop(['horas','datas'], axis=1, inplace=True)
df['datas/horas']=[(datetime.fromisoformat(dias)) for dias in df['datas/horas']]

In [171]:
df

,ss,di,Demanda,datas/horas
0,seco,2,36703.0,2021-09-26 00:00:00
1,seco,2,35673.0,2021-09-26 00:30:00
2,seco,2,34527.0,2021-09-26 01:00:00
3,seco,2,33996.0,2021-09-26 01:30:00
4,seco,2,33526.0,2021-09-26 02:00:00
...,...,...,...,...
216,n,4,5944.0,2021-09-28 00:00:00
217,n,4,5618.0,2021-09-28 07:00:00
218,n,4,6125.0,2021-09-28 10:00:00
219,n,4,6504.0,2021-09-28 22:00:00


<a name="5"></a>
<h2> Parte 5 - Tranformando df para o o formato json pedido</h2>

In [172]:
#Preparando o json

json_branco = []
for index in df.index:
    Data = data_partida
    valores_list = list()
    
    valores_list = [{'Demanda':float(df.loc[index]['Demanda']), 'data':df.loc[index]['datas/horas']}]
    json_branco.append({'subsistema': df.loc[index]['ss'], 'data_partida':Data, 'valores':valores_list})

In [173]:
#Salvando arquivo
json_final = {'resultado':json_branco}
with open('resultado.json', 'w') as outfile:
    json.dump(json_final, outfile, indent=4, default=str)